# `AML — Task 2:` Heart rhythm classification from raw ECG signals
---

In [ ]:
# For TQDM :
#! python3.6 -m pip install ipywidgets
#! python3.6 -m pip install --upgrade jupyter
#! jupyter nbextension enable --py widgetsnbextension

In [ ]:
import numpy as np
import pandas as pd
import biosppy.signals.ecg as ecg
import biosppy.signals.tools as tools
from biosppy.plotting import plot_ecg
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier

---
## Dataset import and export to `csv`

In [ ]:
def load_from_csv(extension="", drop_id = True):
    X_train = pd.read_csv('data/X_train' + extension + '.csv')
    y_train = pd.read_csv('data/y_train' + extension + '.csv')
    X_test  = pd.read_csv('data/X_test' + extension + '.csv')
    
    if drop_id:
        X_train = X_train.drop(columns=['id'])
        y_train = y_train.drop(columns=['id'])
        X_test  = X_test.drop(columns=['id'])
     
    return X_train, y_train, X_test

In [ ]:
def export_to_csv(X_train, y_train, X_test, extension="_cleaned"):
    X_train.to_csv('data/X_train' + extension + '.csv', index=False)
    y_train.to_csv('data/y_train' + extension + '.csv', index=False)
    X_test.to_csv('data/X_test' + extension + '.csv', index=False)

## Submission export to `csv`

In [ ]:
def create_submission(sub_id, pred, basepath='submissions/task2-sub'):
    result = pred.copy().rename(columns={0: 'y'})
    result['id'] = range(0, len(result))
    result = result[['id', 'y']]
    result.to_csv(basepath + str(sub_id) + '.csv', index=False)

---
## Data processing

In [ ]:
def series_to_heartbeats(time_series: pd.Series, sampling_rate=300.0) -> np.array:
    no_nans = time_series.dropna()
    rpeaks = ecg.engzee_segmenter(no_nans, sampling_rate)['rpeaks']
    beats, rpeaks = ecg.extract_heartbeats(no_nans, rpeaks, sampling_rate)
    beats = beats if len(beats.shape) == 2 else beats.reshape((1, -1))
    return beats, rpeaks

### Feature extraction methods

In [ ]:
def extract_rpeak_features(filtered: np.array, rpeaks: np.array):
    rpeaks_amplitudes = [filtered[rpeak] for rpeak in rpeaks]
    rpeaks_mean = np.mean(rpeaks_amplitudes)
    rpeaks_std = np.std(rpeaks_amplitudes)
    return rpeaks, rpeaks_amplitudes, rpeaks_mean, rpeaks_std

In [ ]:
def extract_qpeak_features(filtered: np.array, rpeaks: np.array, window_size=50):
    qpeaks = [rpeak - window_size + np.argmin(filtered[rpeak-window_size:rpeak]) for rpeak in rpeaks]
    qpeaks_amplitudes = [filtered[qpeak] for qpeak in qpeaks]
    qpeaks_mean = np.mean(qpeaks_amplitudes)
    qpeaks_std = np.std(qpeaks_amplitudes)
    return qpeaks, qpeaks_amplitudes, qpeaks_mean, qpeaks_std

In [ ]:
def extract_speak_features(filtered: np.array, rpeaks: np.array, window_size=50):
    speaks = [rpeak + np.argmin(filtered[rpeak:rpeak+window_size]) for rpeak in rpeaks]
    speaks_amplitudes = [filtered[speak] for speak in speaks]
    speaks_mean = np.mean(speaks_amplitudes)
    speaks_std = np.std(speaks_amplitudes)
    return speaks, speaks_amplitudes, speaks_mean, speaks_std

In [ ]:
def extract_qrs_durations_features(qpeaks: np.array, speaks: np.array):
    qrs_durations = [speak - qpeak for qpeak, speak in zip(qpeaks, speaks)]
    qrs_durations_mean = np.mean(qrs_durations)
    qrs_durations_std = np.std(qrs_durations)
    return qrs_durations, qrs_durations_mean, qrs_durations_std

In [ ]:
#TODO: maybe change durations to seconds?
def extract_rr_durations_features(rpeaks: np.array):
    rr_durations = [r2 - r1 for r1, r2 in zip(rpeaks, rpeaks[1:])]
    rr_durations_mean = np.mean(rr_durations)
    rr_durations_std = np.std(rr_durations)
    return rr_durations, rr_durations_mean, rr_durations_std

In [ ]:
def extract_heart_rate_features(heart_rate: np.array):
    heart_rate_mean = np.mean(heart_rate)
    heart_rate_std = np.std(heart_rate)
    return heart_rate_mean, heart_rate_std

### Main feature extraction method

In [ ]:
def extract_features(time_series: pd.Series, sampling_rate=300) -> np.array:
    # Drop nan values in the time series
    no_nans = time_series.dropna()
    
    # Extract main features
    ts, filtered, rpeaks, _, templates, _, heart_rate = ecg.ecg(no_nans, sampling_rate, show=False)
    assert len(rpeaks) > 1, 'ECG cannot have a single R peak'
    assert len(templates) > 1, 'ECG cannot have a single heartbeat'
    
    # Extract Q,R,S peak features
    speaks, qpeaks_amplitudes, qpeaks_mean, qpeaks_std = extract_qpeak_features(filtered, rpeaks)
    rpeaks, rpeaks_amplitudes, rpeaks_mean, rpeaks_std = extract_rpeak_features(filtered, rpeaks)
    qpeaks, speaks_amplitudes, speaks_mean, speaks_std = extract_speak_features(filtered, rpeaks)
    
    # Extract RR, QRS durations features
    rr_durations, rr_durations_mean, rr_durations_std = extract_rr_durations_features(rpeaks)
    qrs_durations, qrs_durations_mean, qrs_durations_std = extract_qrs_durations_features(qpeaks, speaks)
    
    # Extract heart rate features
    if len(heart_rate) == 0:
        heart_rate = rr_durations # temp fix
    heart_rate_mean, heart_rate_std = extract_heart_rate_features(heart_rate)
    
    #TODO: Extract SNR ratio (http://www.cinc.org/archives/2011/pdf/0609.pdf)
    snr = np.quantile(np.std(templates, axis=0), 0.35)
    
    # Use this to go from index differences to seconds
    index_to_time = ts[-1] / len(filtered)
    # Extract pNN28 (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1767394/)
    pNN28 = (np.array(rr_durations) * index_to_time > 0.028).sum() / len(rr_durations)
    
    # Return extracted features
    return pd.Series([rpeaks_mean, 
                      rpeaks_std,
                      rr_durations_mean, 
                      rr_durations_std, 
                      heart_rate_mean, 
                      heart_rate_std, 
                      snr,
                      speaks_mean,
                      speaks_std,
                      qpeaks_mean,
                      qpeaks_std,
                      qrs_durations_mean,
                      qrs_durations_std,
                      pNN28,])

---
## Data standardization

In [ ]:
def standardize_data(X_train, X_test):
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return (X_train_scaled, X_test_scaled)

---
## Models defintions

In [ ]:
def svc(X_train, y_train):
    svc = SVC()
    gs_svc_params = {
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": np.logspace(0, 1, 2),
        "class_weight": ["balanced", None]
    }
    gs_svc = GridSearchCV(svc, gs_svc_params, cv=5, verbose=3, scoring='f1_micro', error_score='raise')
    gs_svc.fit(X_train, y_train)
    
    print(f"The best validation score obtained is {gs_svc.best_score_:.5f} +- \
        {gs_svc.cs_results_['std_test_score'][gs_svc.best_index_]} with\n\t{gs_svc.best_params_}")
    
    return gs_svc

In [ ]:
def random_forest(X_train, y_train):
    random_forest = RandomForestClassifier()
    gs_forest_params = {
     "n_estimators": np.arange(100, 400, 100),
     "max_depth": [None], #np.arange(2, 8, 1),
     "min_samples_split": [2], #np.arange(2, 8, 1),
     "min_samples_leaf": [1], #np.arange(1, 9, 2),
     "class_weight": ["balanced", None],
     "random_state": [0], 
    }
    
    gs_forest = GridSearchCV(random_forest, gs_forest_params, cv=5, verbose=3, 
                             scoring='f1_micro', error_score='raise')
    
    gs_forest.fit(X_train, y_train)

    print(f"The best validation score obtained is {gs_forest.best_score_:.5f} +- \
        {gs_forest.cs_results_['std_test_score'][gs_forest.best_index_]} with\n\t{gs_forest.best_params_}")
    
    return gs_forest

In [ ]:
def gbc(X_train, y_train):
    gbc = GradientBoostingClassifier()
    gs_gbc_params = {
        "loss": ["deviance"],
        "learning_rate": [0.1], # TODO
        "n_estimators": [100], #np.arange(100, 400, 100),
        "subsample": [1], # TODO
        "criterion": ["friedman_mse"],
        "min_samples_split": [2], # TODO
        "min_samples_leaf": [1], # TODO
        "n_iter_no_change": [None], # TODO
        "tol": [1e-4], # TODO
    }
    
    gs_gbc = GridSearchCV(gbc, gs_gbc_params, cv=5, verbose=3, scoring='f1_micro', error_score='raise')
    gs_gbc.fit(X_train, y_train)
    
    print(f"The best validation score obtained is {gs_gbc.best_score_:.5f} +- \
        {gs_gbc.cs_results_['std_test_score'][gs_gbc.best_index_]} with\n\t{gs_gbc.best_params_}")
    
    return gs_gbc

In [ ]:
def ensemble(models, X_train, y_train):
    ensemble = VotingClassifier(estimators=[(str(i), model) for i, model in enumerate(models)])
    gs_ensemble_params = {
     "voting": ["hard", "soft"]
    }
    
    gs_ensemble = GridSearchCV(ensemble, gs_ensemble_params, cv=5, verbose=3, 
                               scoring='f1_micro', error_score='raise')
    
    gs_ensemble.fit(X_train, y_train)

    print(f"The best validation score obtained is {gs_ensemble.best_score_:.5f} +- \
        {gs_ensemble.cs_results_['std_test_score'][gs_ensemble.best_index_]} with\n\t{gs_ensemble.best_params_}")
    
    return gs_ensemble

---
## Main Pipeline

#### Load dataset

In [ ]:
X_train_raw, y_train_raw, X_test_raw = load_from_csv()

In [ ]:
X_train_raw.isna().sum(axis='columns').hist()

In [ ]:
pd.DataFrame(np.mean(series_to_heartbeats(X_train_raw.iloc[0])[0], axis=0)).plot()

#### Extract features

In [ ]:
X_train = X_train_raw.progress_apply(extract_features, axis=1)

In [ ]:
X_test = X_test_raw.progress_apply(extract_features, axis=1)

In [ ]:
print(f"X_train has {X_train.isna().sum().sum()} null values.")
print(f"X_test has {X_test.isna().sum().sum()} null values.")

#### Standardize dataset

In [ ]:
X_train, X_test = standardize_data(X_train, X_test)

In [ ]:
gs_svc = svc(X_train, np.array(y_train_raw).ravel())

In [ ]:
gs_random_forest = random_forest(X_train, np.array(y_train_raw).ravel())

In [ ]:
gs_gbc = gbc(X_train, np.array(y_train_raw).ravel())

In [ ]:
gs_ensemble = ensemble([
    SVC(probability=True, **gs_svc.best_params_), 
    RandomForestClassifier(**gs_random_forest.best_params_),
    GradientBoostingClassifier(**gs_gbc.best_params_)
], X_train, np.array(y_train_raw).ravel())

---
## Generate new submission

In [ ]:
model = gs_ensemble
sub_id = 10
prediction = pd.DataFrame(model.predict(X_test))

In [ ]:
#create_submission(sub_id, prediction)

**Solutions must be submitted on the [project website](https://aml.ise.inf.ethz.ch/task2/).**